In [6]:
import os
ml_script_fl = os.path.join(os.pardir, "src", "models", "machine_learning_api.py")
print(ml_script_fl)
print(os.getcwd())


..\src\models\machine_learning_api.py
D:\ML\DATA_SCIENCE\titanic\notebooks


In [17]:
%%writefile $ml_script_fl

from flask import Flask, request
import pandas as pd
import numpy as np
import os, sys, json, pickle

app = Flask(__name__)

"""Load Model and Scalar files"""
model_file_path = os.path.join(os.path.pardir, os.path.pardir, 'models', 'lr_model.pkl')
scalar_file_path = os.path.join(os.path.pardir, os.path.pardir, 'models', 'lr_scalar.pkl')
mdl_fl_pkl = open(model_file_path,"rb")
scl_fl_pkl = open(scalar_file_path,"rb")

model = pickle.load(mdl_fl_pkl)
scalar = pickle.load(scl_fl_pkl)

columns = [u'Age', u'Fare', u'FamilySize', u'IsMother', u'IsMale',
           u'Deck_A', u'Deck_B', u'Deck_C', u'Deck_D', u'Deck_E',
           u'Deck_F', u'Deck_G', u'Deck_Z', u'Pclass_1', u'Pclass_2',
           u'Pclass_3', u'Title_Lady', u'Title_Master', u'Title_Miss',
           u'Title_Mr', u'Title_Mrs', u'Title_Officer', u'Title_Sir',
           u'Fare_Bin_very_low', u'Fare_Bin_low', u'Fare_Bin_medium',
           u'Fare_Bin_high', u'Embarked_C', u'Embarked_Q', u'Embarked_S',
           u'Agestate_Adults', u'Agestate_Child']

@app.route('/api', methods=['POST'])
def make_predictions():
    data = json.dumps(request.get_json(force=True))
    df = pd.read_json(data)
    passenger_ids = df.PassengerId.ravel()
    survivals = df.Survived.ravel()
    X = df[columns].as_matrix().astype('float')
    Xsc = scalar.transform(X)
    p = model.predict(Xsc)
    df_response = pd.DataFrame({"PassengerId":passenger_ids, "Predicted":p, "Actuals":survivals})
    return df_response.to_json()
    
    name = data['name']
    return "Hello to API World : {0}".format(name)

if __name__ == "__main__":
    app.run(port=8293, debug=True)


Overwriting ..\src\models\machine_learning_api.py


In [18]:
import json, requests
url = "http://127.0.0.1:8292/api"
data = json.dumps({"name":"Debabrata"})
r = requests.post(url,data)
r.text

'Hello to API World : Debabrata'

In [19]:
! ls -lrt ..\src\models\

total 5
-rw-r--r-- 1 debab 197609    0 Dec 30 00:23 predict_model.py
-rw-r--r-- 1 debab 197609    0 Dec 30 00:23 train_model.py
-rw-r--r-- 1 debab 197609  307 Jan  1 20:30 hello_world_api.py
-rw-r--r-- 1 debab 197609 1667 Jan  1 20:58 machine_learning_api.py


### API Invocation 

In [20]:
import os
import pandas as pd
processed_data_path = os.path.join(os.path.pardir, "data", "processed")
tr_fl_pt = os.path.join(processed_data_path, "train.csv")
train_df = pd.read_csv(tr_fl_pt)
sur_pass = train_df[train_df['Survived']==1][:10]



In [13]:
sur_pass

,PassengerId,Survived,Age,Fare,FamilySize,IsMother,IsMale,Deck_A,Deck_B,Deck_C,...,Title_Sir,Fare_Bin_very_low,Fare_Bin_low,Fare_Bin_medium,Fare_Bin_high,Embarked_C,Embarked_Q,Embarked_S,Agestate_Adults,Agestate_Child
1,2,1,38.0,71.2833,2,0,0,0,0,1,...,0,0,0,0,1,1,0,0,1,0
2,3,1,26.0,7.9250,1,0,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
3,4,1,35.0,53.1000,2,0,0,0,0,1,...,0,0,0,0,1,0,0,1,1,0
8,9,1,27.0,11.1333,3,1,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
9,10,1,14.0,30.0708,2,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,1
10,11,1,4.0,16.7000,3,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,1
11,12,1,58.0,26.5500,1,0,0,0,0,1,...,0,0,0,1,0,0,0,1,1,0
15,16,1,55.0,16.0000,1,0,0,0,0,0,...,0,0,0,1,0,0,0,1,1,0
17,18,1,29.0,13.0000,1,0,1,0,0,0,...,0,0,1,0,0,0,0,1,1,0
19,20,1,35.0,7.2250,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,1,0


In [22]:
import requests
def make_api_request(data):
    url = "http://127.0.0.1:8293/api"
    r = requests.post(url,data)
    print(r.status_code)
    return r.json()

from pprint import pprint as pp
pp(make_api_request(sur_pass.to_json()))

200
{'Actuals': {'0': 1,
             '1': 1,
             '2': 1,
             '3': 1,
             '4': 1,
             '5': 1,
             '6': 1,
             '7': 1,
             '8': 1,
             '9': 1},
 'PassengerId': {'0': 2,
                 '1': 11,
                 '2': 12,
                 '3': 16,
                 '4': 18,
                 '5': 20,
                 '6': 3,
                 '7': 4,
                 '8': 9,
                 '9': 10},
 'Predicted': {'0': 1,
               '1': 1,
               '2': 1,
               '3': 1,
               '4': 0,
               '5': 1,
               '6': 1,
               '7': 1,
               '8': 1,
               '9': 1}}


In [23]:
res = make_api_request(train_df.to_json())
df_res = pd.read_json(json.dumps(res))
df_res.head()

200


,Actuals,PassengerId,Predicted
0,0,1,0
1,1,2,1
10,1,108,0
100,0,190,0
101,1,20,1


In [25]:
import numpy as np
np.mean(df_res.Actuals == df_res.Predicted)

0.83838383838383834

In [34]:
from flask import Flask, request
import pandas as pd
import numpy as np
import os, sys, json, pickle
from matplotlib import pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split


"""Load Model and Scalar files"""
model_file_path = os.path.join( os.path.pardir, 'models', 'lr_model.pkl')
scalar_file_path = os.path.join( os.path.pardir, 'models', 'lr_scalar.pkl')
mdl_fl_pkl = open(model_file_path,"rb")
scl_fl_pkl = open(scalar_file_path,"rb")

model = pickle.load(mdl_fl_pkl)
scalar = pickle.load(scl_fl_pkl)

columns = [u'Age', u'Fare', u'FamilySize', u'IsMother', u'IsMale',
           u'Deck_A', u'Deck_B', u'Deck_C', u'Deck_D', u'Deck_E',
           u'Deck_F', u'Deck_G', u'Deck_Z', u'Pclass_1', u'Pclass_2',
           u'Pclass_3', u'Title_Lady', u'Title_Master', u'Title_Miss',
           u'Title_Mr', u'Title_Mrs', u'Title_Officer', u'Title_Sir',
           u'Fare_Bin_very_low', u'Fare_Bin_low', u'Fare_Bin_medium',
           u'Fare_Bin_high', u'Embarked_C', u'Embarked_Q', u'Embarked_S',
           u'Agestate_Adults', u'Agestate_Child']

import os
import pandas as pd
processed_data_path = os.path.join(os.path.pardir, "data", "processed")
tr_fl_pt = os.path.join(processed_data_path, "train.csv")
train_df = pd.read_csv(tr_fl_pt)
sur_pass = train_df[train_df['Survived']==1][:10]
y=train_df['Survived']
X = train_df[columns].as_matrix().astype('float')
Xsc = scalar.transform(X)
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=0)

KeyError: '[0 1 1 1 0 0 0 0 1 1 1 1 0 0 0 1 0 1 0 1 0 1 1 1 0 1 0 0 1 0 0 1 1 0 0 0 1\n 0 0 1 0 0 0 1 1 0 0 1 0 0 0 0 1 1 0 1 1 0 1 0 0 1 0 0 0 1 1 0 1 0 0 0 0 0\n 1 0 0 0 1 1 0 1 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0\n 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0 1 1 0 0 0 1 0\n 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1\n 0 1 1 0 0 1 0 1 1 1 1 0 0 1 0 0 0 0 0 1 0 0 1 1 1 0 1 0 0 0 1 1 0 1 0 1 0\n 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 1\n 1 0 1 0 0 0 0 0 1 1 1 0 1 1 0 1 1 0 0 0 1 0 0 0 1 0 0 1 0 1 1 1 1 0 0 0 0\n 0 0 1 1 1 1 0 1 0 1 1 1 0 1 1 1 0 0 0 1 1 0 1 1 0 0 1 1 0 1 0 1 1 1 1 0 0\n 0 1 0 0 1 1 0 1 1 0 0 0 1 1 1 1 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 1 1 1\n 1 0 0 0 0 1 1 0 0 0 1 1 0 1 0 0 0 1 0 1 1 1 0 1 1 0 0 0 0 1 1 0 0 0 0 0 0\n 1 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 1 1 1 0 0 1 0 1 0 0 1 0 0 1\n 1 1 1 1 1 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 0\n 

In [48]:
import os
import pandas as pd
processed_data_path = os.path.join(os.path.pardir, "data", "processed")
tr_fl_pt = os.path.join(processed_data_path, "train.csv")
train_df = pd.read_csv(tr_fl_pt)
sur_pass = train_df[train_df['Survived']==1][:10]
y=train_df['Survived']
X = train_df[columns].as_matrix().astype('float')
Xsc = scalar.transform(X)
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=0)
nn = MLPClassifier(activation='logistic', solver='sgd', hidden_layer_sizes=(100,50),random_state=1)
nn.fit(Xtrain, ytrain)
pred=nn.predict(Xtest)
z=pd.DataFrame(pred)
e=pd.concat([z,ytest],axis=1)
e.head()

C:\Users\debab\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


,0,Survived
0,0.0,NaN
1,0.0,1.0
2,0.0,NaN
3,0.0,NaN
4,0.0,NaN


In [30]:
def make_predictions():
    data = train_df.to_json()
    df = pd.read_json(data)
    passenger_ids = df.PassengerId.ravel()
    survivals = df.Survived.ravel()
    X = df[columns].as_matrix().astype('float')
    Xsc = scalar.transform(X)
    p = model.predict(Xsc)
    df_response = pd.DataFrame({"PassengerId":passenger_ids, "Predicted":p, "Actuals":survivals})
    return df_response

In [33]:
x=make_predictions()
x[["Actuals","Predicted","PassengerId"]]

,Actuals,Predicted,PassengerId
0,0,0,1
1,1,1,2
2,1,1,11
3,0,1,101
4,0,0,102
5,0,1,103
6,0,0,104
7,0,0,105
8,0,0,106
9,1,1,107
